### Start

In [1]:
import torch
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.llms import Ollama
from langchain.agents import AgentExecutor, Tool
from langchain.agents import initialize_agent, AgentType
from concurrent.futures import ThreadPoolExecutor
import json

# Enable MPS (M1 GPU) if available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [2]:
device

device(type='mps')

In [ ]:
# Initialize Ollama with the llama3.2 model
llm = Ollama(model="llama3.2")

In [4]:
# Agent 1: Schedule Generator
def schedule_generator(date, temp_high, temp_low, weather, air_quality, attractions):
    prompt = f"""
    You are an AI assistant specializing in creating schedules for travelers in Busan. Based on the following weather, temperature, air quality, and attractions, generate a day schedule that maximizes enjoyment while considering environmental factors.

    Date: {date}
    Temperature High: {temp_high}
    Temperature Low: {temp_low}
    Weather: {weather}
    Air Quality: {air_quality}
    Attractions: {attractions}

    Please create a detailed schedule including recommended visit times, breaks, and any special considerations due to the weather and air quality.
    """
    return llm(prompt)

# Agent 2: Recommendation Generator
def recommendation_generator(date, temp_high, temp_low, weather, air_quality, attractions):
    prompt = f"""
    You are an AI assistant providing recommendations based on the following environmental factors and attractions in Busan. Considering the weather, air quality, and attractions available, provide recommendations for the day, including tips for dealing with adverse conditions and optimizing the experience.

    Date: {date}
    Temperature High: {temp_high}
    Temperature Low: {temp_low}
    Weather: {weather}
    Air Quality: {air_quality}
    Attractions: {attractions}

    Provide a detailed recommendation that includes which attractions to visit, precautions, and how to manage the day in light of the environmental conditions.
    """
    return llm(prompt)


In [ ]:
# Define tools
tools = [
    Tool(
        name="Schedule Generator",
        func=schedule_generator,
        description="Use this tool to generate a detailed schedule for the day."
    ),
    Tool(
        name="Recommendation Generator",
        func=recommendation_generator,
        description="Use this tool to provide detailed recommendations based on weather and conditions."
    )
]

# Initialize the agent
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True
)

# Data  example for the itinerary: We can use API to get the data
data = [
    {
        "date": "2024-10-05",
        "temp_high": 30,
        "temp_low": 14,
        "weather": "Windy",
        "air_quality": "Hazardous",
        "attractions": [
            {"name": "Haeundae Beach", "recommended_time": 2.98},
            {"name": "Gwangalli Beach", "recommended_time": 3.18},
            {"name": "Taejongdae", "recommended_time": 3.63},
            {"name": "Beomeosa Temple", "recommended_time": 2.4},
            {"name": "Gamcheon Culture Village", "recommended_time": 1.0}
        ]
    }
]

/var/folders/b5/q_9jl9rj403b681rxbdcpgzh0000gn/T/ipykernel_90641/97325001.py:16: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(


In [ ]:
# Function to process a single prompt
def process_prompt(agent, date, temp_high, temp_low, weather, air_quality, attractions):
    formatted_attractions = ", ".join([f"{attr['name']} ({attr['recommended_time']} hrs)" for attr in attractions])
    try:
        # Create input string
        input_str = f"""Generate a schedule and recommendations for a day in Busan on {date} with high temperature {temp_high}°C, low temperature {temp_low}°C, {weather} weather, {air_quality} air quality, and the following attractions: {formatted_attractions}. 
        First, use the Schedule Generator to create a detailed itinerary. Then, use the Recommendation Generator to provide tips and precautions based on the conditions."""
        
        # Run the agent
        result = agent.run(input_str)
        return result
    except Exception as e:
        return f"Error processing prompt: {e}"

In [7]:
# Process each entry in the data
for entry in data:
    result = process_prompt(agent, entry["date"], entry["temp_high"], entry["temp_low"], entry["weather"], entry["air_quality"], entry["attractions"])
    print(f"Generated Output for {entry['date']}:\n{result}")

/var/folders/b5/q_9jl9rj403b681rxbdcpgzh0000gn/T/ipykernel_90641/2586510902.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run(input_str)




> Entering new AgentExecutor chain...
Based on your input, I've created a schedule for a day in Busan on 2024-10-05.

**Detailed Itinerary:**

1. 6:00 AM - Wake up and have breakfast at a nearby café or restaurant.
2. 7:00 AM - Leave for Beomeosa Temple (2.4 hours from your accommodation). Take the bus (approx. 30 minutes) or drive (depending on your preference).
3. 9:24 AM - Arrive at Beomeosa Temple and explore the temple grounds, including the Hall of Jade, the Buddha Statue, and the surrounding gardens.
4. 11:00 AM - Leave for Gamcheon Culture Village (1 hour from Beomeosa Temple). Take a short walk or drive depending on your preference.
5. 12:00 PM - Arrive at Gamcheon Culture Village and explore its colorful streets, shops, and cafes. Don't forget to take some great photos!
6. 1:30 PM - Have lunch at one of the many restaurants in Gamcheon Culture Village or head back to your accommodation for a meal.
7. 2:30 PM - Leave for Haeundae Beach (2 hours from Gamcheon Culture Village)